In [24]:
import os
import numpy as np
from sippy import functionset as fset
import torch.nn.functional as F

from simba.model import Simba
from simba.functions import generate_random_system, generate_data, add_noise, identify_baselines, matlab_baselines
from simba.util import print_all_perf, eval_simba, fix_seed, save_results, check_and_initialize_data

from simba.parameters import base_parameters, baselines_to_use
parameters = base_parameters

In [25]:
# To modify
seed = 123
parameters['max_epochs'] = 3
parameters['init_epochs'] = 3
parameters['init_from_matlab_or_ls'] = False
parameters['device'] = 'cpu'
directory = os.path.join('saves', f'Test_{seed}')

process_noise = False
process_noise_scale = 0.1
output_noise_scale = 0.25

In [26]:
fix_seed(seed)

# Data parameters
number_trajectories = 1
number_trajectories_test = 1
nx = 7
nu = 6
ny = 5

N = 500
dt = 0.05
min_eigenvalue = 0.5


# Simba parameters for this simulation
parameters['input_output'] = True
parameters['id_D'] = True
parameters['stable_A'] = True
parameters['LMI_A'] = True
parameters['init_from_matlab_or_ls'] = False

parameters['grad_clip'] = 100
parameters['learning_rate'] = 0.001
parameters['dropout'] = 0.2
parameters['batch_size'] = 128 

parameters['learn_x0'] = False

parameters['print_each'] = 2500

# The system is a random DISCRETE system, so we set delta = None since A is not
# expected to be identified from a continuous system, i.e., with form I + delta*A
parameters['delta'] = None

path_to_matlab = parameters['path_to_matlab']

In [27]:
# # Prepare the data
# flag = True
# while flag:
#     A, B, C, D = generate_random_system(nx=nx, nu=nu, ny=ny, N=N, stable_A=parameters['stable_A'], min_eigenvalue=min_eigenvalue)
#     # Ensure D is zero
#     D = np.zeros_like(D)
#     # Assume we can observe the first ny states exactly
#     C = np.concatenate([np.eye(ny), np.zeros((ny, nx-ny))], axis=1)
#     true_mask_C = (C > 0) * 1.
#     # Assume the last nu states are controllable, but with unknown parameters
#     true_mask_B = np.concatenate([np.zeros((nx-nu, nu)), np.eye(nu)], axis=0)
#     B = B * true_mask_B
#     # Assume A is tridiagonal, i.e., the states are arranged on a line and only impacted by neighboring states
#     A = np.triu(A,-1) + np.tril(A,1) - A
#     true_mask_A = (A != 0) * 1.
#     # Ensure the modified system is still stable
#     if max(np.abs(np.linalg.eigvals(A))) < 1:
#         flag = False
        
flag = True
while flag:
    A, B, C, D = generate_random_system(nx=nx, nu=nu, ny=ny, N=N, stable_A=parameters['stable_A'], min_eigenvalue=min_eigenvalue)

    true_mask_D = (np.random.random(D.shape) < 0.4) * 1.
    true_mask_C = (np.random.random(C.shape) < 0.4) * 1.
    true_mask_B = (np.random.random(B.shape) < 0.4) * 1.
    true_mask_A = (np.random.random(A.shape) < 0.4) * 1.
    
    D = D * true_mask_D
    C = C * true_mask_C
    B = B * true_mask_B
    A = A * true_mask_A
    
    # Ensure the modified system is still stable
    if max(np.abs(np.linalg.eigvals(A))) < 1:
        flag = False

x0 = np.zeros((number_trajectories,1,nx))
x0_val = x0
x0_test = np.zeros((number_trajectories_test,1,nx))
U = np.zeros((number_trajectories,N,nu))
U_val = np.zeros((number_trajectories,N,nu))
U_test = np.zeros((number_trajectories_test,N,nu))
X = np.zeros((number_trajectories,N,nx))
X_val = np.zeros((number_trajectories,N,nx))
X_test = np.zeros((number_trajectories_test,N,nx))
Y = np.zeros((number_trajectories,N,ny))
Y_val = np.zeros((number_trajectories,N,ny))
Y_test = np.zeros((number_trajectories_test,N,ny))

for t in range(number_trajectories):
    # Creating exciting input sequences
    for i in range(nu):
        U[t, :,i],_,_ =  fset.GBN_seq(N, 0.1)
        U_val[t, :,i],_,_ =  fset.GBN_seq(N, 0.1)
    # Simulate the system to create the data
    U[t,:,:], Y[t,:,:], X[t,:,:] = generate_data(A, B, C, D, N, parameters['id_D'], U=U[t,:,:], x0=x0[t,:,:], gaussian_U=False, process_noise_scale=process_noise_scale, dt=dt)
    U_val[t,:,:], Y_val[t,:,:], X_val[t,:,:] = generate_data(A, B, C, D, N, parameters['id_D'], U=U_val[t,:,:], x0=x0_val[t,:,:], gaussian_U=False, process_noise_scale=process_noise_scale, dt=dt)

for t in range(number_trajectories_test):
    # Creating exciting input sequences
    for i in range(nu):
        U_test[t, :,i],_,_ =  fset.GBN_seq(N, 0.1)
    # Simulate the system to create the data
    U_test[t,:,:], Y_test[t,:,:], X_test[t,:,:] = generate_data(A, B, C, D, N, parameters['id_D'], U=U_test[t,:,:], x0=x0_test[t,:,:], gaussian_U=False, process_noise_scale=process_noise_scale, dt=dt)

# Add output noise if wanted
if output_noise_scale > 0:
    Y = add_noise(Y, voss=False, colored=False, scale=output_noise_scale)

# Store all the parameters for reproducibility
sim_params = (seed, process_noise_scale, output_noise_scale, number_trajectories, number_trajectories_test)
data_params = (A, B, C, D, N, parameters['id_D'])
data = (U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)

In [28]:
# Evaluate classical sysID baselines
U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test = check_and_initialize_data(U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test,
                                                                                                            verbose=parameters['verbose'], autonomous=parameters['autonomous'], 
                                                                                                            input_output=parameters['input_output'], device=parameters['device'])

names, baselines, times, train_ids, validation_ids,test_ids = identify_baselines(nx=nx, U=U, U_val=U_val, U_test=U_test, Y=Y, Y_val=Y_val, Y_test=Y_test,
                                                                                 x0=x0, x0_val=x0_val, x0_test=x0_test, dt=dt,
                                                                                 parameters=parameters, baselines_to_use=baselines_to_use)

print_all_perf(names, times, train_ids, validation_ids, test_ids, Y, Y_val, Y_test)

save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
             validation_ids=validation_ids, test_ids=test_ids, data=data, sim_params=sim_params, data_params=data_params)


Method		Time		Train Perf	Val Perf	Test perf
------------------------------------------------------------------------
ARMAX-ILLS	9.83E-01s	9.45E-01	4.59E+142	7.66E+140
ARMAX-RLLS	1.00E+00s	6.57E-01	1.14E+00	1.23E+00
ARX-ILLS	4.04E-01s	9.39E-01	1.13E+00	1.20E+00
ARX-RLLS	5.29E-01s	9.39E-01	1.13E+00	1.20E+00
OE-ILLS		7.97E-01s	7.29E-01	1.20E+00	1.27E+00
N4SID		1.96E-01s	5.78E-01	6.12E-01	6.38E-01
MOESP		1.16E-01s	6.07E-01	6.26E-01	6.57E-01
CVA		1.31E-01s	5.90E-01	5.91E-01	6.44E-01
PARSIM-K	1.15E+00s	4.63E-01	4.43E-01	5.22E-01
PARSIM-S	7.15E-01s	5.44E-01	5.66E-01	6.42E-01
PARSIM-P	1.21E+00s	5.57E-01	5.97E-01	6.67E-01
------------------------------------------------------------------------
mat-ARX		2.89E+00s	3.83E-01	4.40E-01	4.59E-01
mat-N4SID	1.89E+00s	1.57E+00	1.66E+00	1.82E+00
mat-PEM		1.48E+00s	1.04E+00	1.10E+00	1.23E+00


In [5]:
# General SIMBa without prior knowledge
fix_seed(seed)
name = 'SIMBa'
parameters['ms_horizon'] = None
parameters['mask_A'] = None
parameters['learn_A'] = True
parameters['A_init'] = None
parameters['mask_B'] = None
parameters['learn_B'] = True
parameters['B_init'] = None
parameters['mask_C'] = None
parameters['learn_C'] = True
parameters['C_init'] = None
parameters['mask_D'] = None
parameters['learn_D'] = True
parameters['D_init'] = None

simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test)
simba.save(directory=directory, save_name=name)

names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids, 
                                                    U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
            validation_ids=validation_ids, test_ids=test_ids, data=data)


Initilization starts, fitting A!
Epoch	Fitting loss
1	1.95E-01
Total initialization time:	00"
Best loss at epoch 2:	1.94E-01

SIPPY performance (only measured on the first trajectory if there are several for now):
Training loss	Validation loss
3.54E-01	4.69E-01

Training of SIMBa starts!
Training data shape:	(1, 500, *)
Validation data shape:	(1, 500, *)

Epoch	Train loss	Val loss	Test loss
1	3.16E+01	3.95E+01	3.70E+01

Total training time:		00"

Best model performance:
3	3.15E+01	3.91E+01	3.66E+01


In [ ]:
# Assuming we know the structure (sparsity mask) of D
fix_seed(seed)
name = 'SIMBa_mD'
parameters['mask_A'] = None
parameters['learn_A'] = True
parameters['A_init'] = None
parameters['mask_B'] = None
parameters['learn_B'] = True
parameters['B_init'] = None
parameters['mask_C'] = None
parameters['learn_C'] = True
parameters['C_init'] = None
parameters['mask_D'] = true_mask_D
parameters['learn_D'] = True
parameters['D_init'] = None


simba = Simba(nx=nx, nu=nu, ny=ny, delta=delta, parameters=parameters)
simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test)
simba.save(directory=directory, save_name=name)

names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids, 
                                                    U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
            validation_ids=validation_ids, test_ids=test_ids, data=data)

In [7]:
# Assuming we know the structure (sparsity mask) of C
fix_seed(seed)
name = 'SIMBa_mC'
parameters['mask_A'] = None
parameters['learn_A'] = True
parameters['A_init'] = None
parameters['mask_B'] = None
parameters['learn_B'] = True
parameters['B_init'] = None
parameters['mask_C'] = true_mask_C
parameters['learn_C'] = True
parameters['C_init'] = None
parameters['mask_D'] = None
parameters['learn_D'] = False
parameters['D_init'] = D


simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test)
simba.save(directory=directory, save_name=name)

names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids, 
                                                    U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
            validation_ids=validation_ids, test_ids=test_ids, data=data)


Initilization starts, fitting A!
Epoch	Fitting loss
1	1.68E-01
Total initialization time:	00"
Best loss at epoch 2:	1.67E-01

SIPPY performance (only measured on the first trajectory if there are several for now):
Training loss	Validation loss
3.54E-01	4.69E-01

Training of SIMBa starts!
Training data shape:	(1, 500, *)
Validation data shape:	(1, 500, *)

Epoch	Train loss	Val loss	Test loss
1	3.04E+01	4.12E+01	3.84E+01

Total training time:		00"

Best model performance:
3	3.14E+01	4.11E+01	3.84E+01


In [8]:
# Assuming we know the true matrix C
fix_seed(seed)
name = 'SIMBa_C'
parameters['mask_A'] = None
parameters['learn_A'] = True
parameters['A_init'] = None
parameters['mask_B'] = None
parameters['learn_B'] = True
parameters['B_init'] = None
parameters['mask_C'] = None
parameters['learn_C'] = False
parameters['C_init'] = C

simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test)
simba.save(directory=directory, save_name=name)

names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids, 
                                                    U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
            validation_ids=validation_ids, test_ids=test_ids, data=data)


Initilization starts, fitting A!
Epoch	Fitting loss
1	1.90E-01
Total initialization time:	00"
Best loss at epoch 2:	1.89E-01

SIPPY performance (only measured on the first trajectory if there are several for now):
Training loss	Validation loss
3.54E-01	4.69E-01

Training of SIMBa starts!
Training data shape:	(1, 500, *)
Validation data shape:	(1, 500, *)

Epoch	Train loss	Val loss	Test loss
1	3.22E+01	4.21E+01	3.93E+01

Total training time:		00"

Best model performance:
3	3.32E+01	4.20E+01	3.93E+01


In [9]:
# Assuming we know C and the structure (sparsity mask) of B
fix_seed(seed)
name = 'SIMBa_CmB'
parameters['mask_A'] = None
parameters['learn_A'] = True
parameters['A_init'] = None
parameters['mask_B'] = true_mask_B
parameters['learn_B'] = True
parameters['B_init'] = None
parameters['mask_C'] = None
parameters['learn_C'] = False
parameters['C_init'] = C

simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test)
simba.save(directory=directory, save_name=name)

names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids, 
                                                    U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
            validation_ids=validation_ids, test_ids=test_ids, data=data)


Initilization starts, fitting A!
Epoch	Fitting loss
1	1.38E-01
Total initialization time:	00"
Best loss at epoch 2:	1.37E-01

SIPPY performance (only measured on the first trajectory if there are several for now):
Training loss	Validation loss
3.54E-01	4.69E-01

Training of SIMBa starts!
Training data shape:	(1, 500, *)
Validation data shape:	(1, 500, *)

Epoch	Train loss	Val loss	Test loss
1	3.32E+01	4.21E+01	3.93E+01

Total training time:		00"

Best model performance:
3	3.28E+01	4.21E+01	3.93E+01


In [10]:
# Assuming we know C and B
fix_seed(seed)
name = 'SIMBa_CB'
parameters['mask_A'] = None
parameters['learn_A'] = True
parameters['A_init'] = None
parameters['mask_B'] = None
parameters['learn_B'] = False
parameters['B_init'] = B
parameters['mask_C'] = None
parameters['learn_C'] = False
parameters['C_init'] = C

simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test)
simba.save(directory=directory, save_name=name)

names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids, 
                                                    U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
            validation_ids=validation_ids, test_ids=test_ids, data=data)


Initilization starts, fitting A!
Epoch	Fitting loss
1	1.83E-01
Total initialization time:	00"
Best loss at epoch 2:	1.81E-01

SIPPY performance (only measured on the first trajectory if there are several for now):
Training loss	Validation loss
3.54E-01	4.69E-01

Training of SIMBa starts!
Training data shape:	(1, 500, *)
Validation data shape:	(1, 500, *)

Epoch	Train loss	Val loss	Test loss
1	3.17E+01	4.22E+01	3.95E+01

Total training time:		00"

Best model performance:
3	3.22E+01	4.22E+01	3.95E+01


In [11]:
# Assuming we know C, B and the structure of A
fix_seed(seed)
name = 'SIMBa_CBmA'
parameters['mask_A'] = true_mask_A
parameters['learn_A'] = True
parameters['A_init'] = None
parameters['mask_B'] = None
parameters['learn_B'] = False
parameters['B_init'] = B
parameters['mask_C'] = None
parameters['learn_C'] = False
parameters['C_init'] = C

simba = Simba(nx=nx, nu=nu, ny=ny, parameters=parameters)
simba.fit(U, U_val=U_val, U_test=U_test, X=X, X_val=X_val, X_test=X_test, Y=Y, Y_val=Y_val, Y_test=Y_test, x0=x0, x0_val=x0_val, x0_test=x0_test)
simba.save(directory=directory, save_name=name)

names, times, train_ids, validation_ids, test_ids = eval_simba(simba, name, names, times, train_ids, validation_ids, test_ids, 
                                                    U, U_val, U_test, X, X_val, X_test, Y, Y_val, Y_test, x0, x0_val, x0_test)
save_results(directory=directory, save_name='Results', names=names, times=times, train_ids=train_ids, 
            validation_ids=validation_ids, test_ids=test_ids, data=data)


Initilization starts, fitting A!
Epoch	Fitting loss
1	1.08E-01
Total initialization time:	00"
Best loss at epoch 2:	1.08E-01

SIPPY performance (only measured on the first trajectory if there are several for now):
Training loss	Validation loss
3.54E-01	4.69E-01

Training of SIMBa starts!
Training data shape:	(1, 500, *)
Validation data shape:	(1, 500, *)

Epoch	Train loss	Val loss	Test loss
1	3.03E+01	3.97E+01	3.70E+01

Total training time:		00"

Best model performance:
3	3.01E+01	3.97E+01	3.70E+01


In [15]:
print_all_perf(names, times, train_ids, validation_ids, test_ids, Y, Y_val, Y_test)


Method		Time		Train Perf	Val Perf	Test perf
------------------------------------------------------------------------
ARMAX-ILLS	9.93E-01s	1.23E-01	8.73E-02	8.78E-02
ARMAX-RLLS	9.35E-01s	8.60E-02	8.69E-02	8.66E-02
ARX-ILLS	4.12E-01s	1.30E-01	8.33E-02	8.35E-02
ARX-RLLS	4.21E-01s	1.30E-01	8.33E-02	8.35E-02
OE-ILLS		6.28E-01s	1.39E-01	8.29E+00	8.13E+00
N4SID		8.19E-02s	4.56E-01	6.08E-01	5.23E-01
MOESP		1.05E-01s	4.84E-01	6.16E-01	5.31E-01
CVA		1.15E-01s	3.33E-01	5.29E-01	4.38E-01
PARSIM-K	6.38E-01s	3.54E-01	4.69E-01	4.18E-01
PARSIM-S	4.09E-01s	5.57E+00	4.97E+00	4.58E+00
PARSIM-P	1.03E+00s	5.89E+00	5.35E+00	4.93E+00
------------------------------------------------------------------------
mat-ARX		2.79E+00s	1.73E-01	2.66E-01	2.04E-01
mat-N4SID	1.53E+00s	1.14E+00	2.24E+00	2.15E+00
mat-PEM		1.43E+00s	1.70E-01	3.31E-01	2.48E-01
------------------------------------------------------------------------
SIMBa		4.10E-01s	3.93E+01	3.91E+01	3.66E+01
SIMBa_MS	1.75E-01s	3.83E+01	3.88E+01	3.62E+01
SIMBa